In [3]:
pip install pandas_datareader

  Using cached pandas_datareader-0.9.0-py3-none-any.whl (107 kB)
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from pandas_datareader import data
import datetime

In [6]:
data.DataReader?

In [62]:
#only have data for business days, March 5th and 6th missing - weekends
start=datetime.datetime(2015,3,1)
end=datetime.datetime(2016,3,10)


df=data.DataReader(name="GOOG",data_source="yahoo",start=start,end=end)
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-03-02,570.583435,557.220154,558.995300,569.775696,2129600.0,569.775696
2015-03-03,573.814575,564.968872,568.888123,572.069397,1704700.0,572.069397
2015-03-04,575.529907,566.454773,570.304260,571.800110,1876800.0,571.800110
2015-03-05,576.327698,571.840027,573.445618,573.754761,1389600.0,573.754761
2015-03-06,575.101074,565.208191,573.305969,566.130676,1659100.0,566.130676
...,...,...,...,...,...,...
2016-03-04,716.489990,706.020020,714.989990,710.890015,1972100.0,710.890015
2016-03-07,708.091003,686.900024,706.900024,695.159973,2985100.0,695.159973
2016-03-08,703.789978,685.340027,688.590027,693.969971,2076300.0,693.969971


In [63]:
#Building Candlestick Data with Bokeh
from bokeh.plotting import figure, show, output_file

In [64]:
#Find rows where close > open
date_increase=df.index[df.Close>df.Open]
date_decrease=df.index[df.Close<df.Open]

In [65]:
print(type(date_increase))
print(date_increase)

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05',
               '2015-03-09', '2015-03-12', '2015-03-16', '2015-03-18',
               '2015-03-24', '2015-03-30',
               ...
               '2016-02-09', '2016-02-11', '2016-02-17', '2016-02-19',
               '2016-02-24', '2016-02-25', '2016-03-01', '2016-03-08',
               '2016-03-09', '2016-03-10'],
              dtype='datetime64[ns]', name='Date', length=125, freq=None)


In [66]:
def inc_dec(c,o):
    if c > o:
        value="Increase"
    elif c < o:
        value="Decrease"
    else:
        value="Equal"
    return value


df["Status"]=[inc_dec(c,o) for c, o in zip(df.Close,df.Open)]

df["Middle"]=(df.Open+df.Close)/2
df["Height"]=abs(df.Close-df.Open)

In [67]:
df

,High,Low,Open,Close,Volume,Adj Close,Status,Middle,Height
Date,,,,,,,,,
2015-03-02,570.583435,557.220154,558.995300,569.775696,2129600.0,569.775696,Increase,564.385498,10.780396
2015-03-03,573.814575,564.968872,568.888123,572.069397,1704700.0,572.069397,Increase,570.478760,3.181274
2015-03-04,575.529907,566.454773,570.304260,571.800110,1876800.0,571.800110,Increase,571.052185,1.495850
2015-03-05,576.327698,571.840027,573.445618,573.754761,1389600.0,573.754761,Increase,573.600189,0.309143
2015-03-06,575.101074,565.208191,573.305969,566.130676,1659100.0,566.130676,Decrease,569.718323,7.175293
...,...,...,...,...,...,...,...,...,...
2016-03-04,716.489990,706.020020,714.989990,710.890015,1972100.0,710.890015,Decrease,712.940002,4.099976
2016-03-07,708.091003,686.900024,706.900024,695.159973,2985100.0,695.159973,Decrease,701.029999,11.740051
2016-03-08,703.789978,685.340027,688.590027,693.969971,2076300.0,693.969971,Increase,691.279999,5.379944


In [88]:


p=figure(x_axis_type="datetime",width=1000,height=300,sizing_mode="scale_width")
p.title.text="Candlestick Chart"
p.grid.grid_line_alpha=0.6

hours_12=12*60*60*1000

p.segment(df.index,df.High,df.index,df.Low,color="Black")

p.rect(df.index[df.Status=="Increase"],df.Middle[df.Status=="Increase"],
       hours_12,df.Height[df.Status=="Increase"],
       fill_color="#CCFFFF",line_color="black")

p.rect(df.index[df.Status=="Decrease"],df.Middle[df.Status=="Decrease"],
       hours_12,df.Height[df.Status=="Decrease"],
       fill_color="#FF3333",line_color="black")



output_file("CS.html")
show(p)